# Pandas - import/eksport danych

## Bezpośredni zapis do pliku

Każdą strukturę danych (i nie tylko) utworzoną w Pythonie możemy zrzucić do pliku za pomocą funkcji biblioteki `pickle`. Piklowanie konwertuje dane do ciągu znaków (serializacja), z których skrypt Pythona jest w stanie bezpośrednio je odtworzyć, bez potrzeby konwersji. Jeżeli mamy strukturę `Series` lub `DataFrame`, to funkcja piklowania jest następująca: `struktura.to_pickle('ścieżka dostępu', compression='infer', protocol=4)`, gdzie:

- `'ścieżka_dostępu'` - ścieżka, pod którą chcemy zapisać plik wraz z nazwą pliku. Jeżeli zostawiamy domyślną opcję `compression='infer'` a chcemy od razu skompresować plik, to podajemy odpowiednie rozszerzenie: ` .gzip, .bz2, .zip, .xz`. Jeżeli podamy inne rozszerzenie pliku (popularne opcje ` .txt, .pickle`) lub nie podamy rozszerzenia w ogóle (Windows wymaga podania rozszerzenia), to plik nie zostanie skompresowany.
- `compression` - typ kompresji użytej przy tworzeniu pliku. Do wyboru:
    - `'infer'` (domyślny) - typ kompresji (o ile w ogóle ma być użyta) zostanie dobrany po rozszerzeniu pliku,
    - `‘gzip’, ‘bz2’, ‘zip’, ‘xz’` - zadany typ kompresji,
    - `None` - bez kompresji.
- `protocol` (domyślny: najwyższy dostępny) - typ protokołu użytego do serializacji danych. Do wyboru: `0,1,2,3,4`. Wersja 4 działa od Pythona 3.4, wersja 3 od 3.0, wersja 2 od 2.3, 1 i 0 działają zawsze. Im wyższa wersja, tym efektywniejsza serializacja (możliwość piklowania dużych obiektów, optymalizacja ze względu na typ danych itp.). Jeżeli podamy liczbę ujemną, użyty zostanie najwyższy dostępny protokół.

Funkcja odczytu zapiklowanej struktury danych biblioteki Pandas to: `pandas.read_pickle('ścieżka dostępu', compression='infer')`.

Przy domyślnych ustawieniach (`compression='infer'`) jeżeli plik był skompresowany, to zostanie rozpakowany metodą dobraną na podstawie jego rozszerzenia. Jeżeli plik nie posiada rozszerzenia a wiemy, że był skompresowany konkretną metodą, to zadajemy metodę `‘gzip’, ‘bz2’, ‘zip’` lub `‘xz’`. Jeżeli plik nie był skompresowany a posiada rozszerzenie sugerujące kompresję (jest to możliwe, np. gdy plik został zapiklowany z rozszerzeniem `.zip` przy ustawieniu `compression=None`, ale generalnie tak się nie robi), to należy podać `compression=None`.

Typ odczytanej struktury będzie dokładnie taki, jak przed zapiklowaniem, tzn. jeżeli zapiklowaliśmy szereg `Series`, to po odczytaniu otrzymamy typ `Series`. Jeżeli chcemy sprawdzić typ zwróconej struktury, to używamy funkcji `type()`. Nie jest dobrym pomysłem wypisywanie na ekran danych nieznanego rozmiaru i typu.

Przykład - piklowanie szeregu i ramki danych:

In [1]:
import pandas as pd

szereg=pd.Series(range(0,100),index=range(1,101))
ramka=pd.DataFrame({'kol1':{'w1':'a','w2':'b','w3':'c'},'kol2':{'w1':1,'w2':2,'w3':3}})

szereg.to_pickle('dane01.zip')
ramka.to_pickle('dane02.zip')

Odczyt pikli:

In [2]:
import pandas as pd

data1=pd.read_pickle('dane01.zip')
data2=pd.read_pickle('dane02.zip')
    
print(type(data1)) 
print(type(data2))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


## Import/eksport z pliku csv

Biblioteka Pandas posiada wbudowane funkcje do importu/eksportu danych w różnych formatach do odpowiednich struktur (`Series`, `DataFrame`). Obsługiwane typy:

- pliki tekstowe typu `coma-separated values` (` .csv`),
- pliki tekstowe typu `fixed-width` (dane w każdym wierzu ustalonej kolumny mają dokładnie taką samą liczbę znaków),
- arkusze kalkulacyjne Excel (` .xls, .xlsx`),
- bazy danych SQL (` .sql`),
- dane w formacie JSON (` .json`), HTML (` .html`), HDF4 i HDF5 (` .hdf, .hdf5, .h5, .hdf4, .h4`),
- pliki binarne w formacie Feather (` .feather`),
- dane w formacie Apache Parquet (` .parquet`),
- dane ładowane bezpośrednio z Google BigQuery,
- pliki wyjściowe programów SAS (` .sas`) i STATA (` .ado, .gph`).

Do importu struktury danych z pliku csv służy funkcja

`pandas.read_csv(filepath_or_buffer, sep=', ', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)`

Opcje:
- `filepath_or_buffer` - ścieżka dostępu do pliku, lub adres URL, np. `'file://localhost/path/to/table.csv'`,
- `sep`, `delimiter` - string, domyślnie `','`; separator kolumn użyty w importowanym pliku. Jeżeli separator w pliku jest inny (np. tab lub spacja), to Python potrafi to wykryć automatycznie (przed zrzuceniem do struktury danych Python parsuje plik csv funkcją `csv.Sniffer`), dlatego spokojnie można pozostawić tę opcję domyślną przy ustawieniu `engine='python'`.
- `header` - `None`, `'infer'`, liczba całkowita lub ich lista, domyślnie `'infer'`; w przypadku pojedynczej liczby jest to numer wiersza pliku csv, w którym znajdują się nazwy kolumn. Zrzucanie pliku do struktury danych rozpocznie się od tego wiersza - poprzednie zostaną zignorowane. Uwaga: `header=0` oznacza, że pierwszy wiersz pliku zawiera nazwy kolumn. Jeżeli w pliku nie ma nazw kolumn w ogóle, należy zadać `header=None`. `header='infer'` wykrywa tę różnicę automatycznie i ustawia albo 0, albo `None`. W przypadku danych multiindeksowanych zadajemy listę numerów wierszy, w których znajdują się indeksy w kolejności od ogólnych do szczególnych.
- `names` - lista, tablica jednowymiarowa; lista nowych nazw kolumn (można też przy okazji dodać puste kolumny). Jeżeli plik nie zawierał nazw kolumn a chcemy je nadać używając tej opcji, to powinniśmy ustawić `header=None` zamiast opcji domyślnej. Jeżeli chcemy nadpisać istniejące nazwy, to powinniśmy ustawić `header=n`, gdzie `n` to numer wiersza zawierającego aktualne nazwy kolumn w pliku.
- `index_col` - liczba, lista liczb lub `False`; numer kolumny, w której znajdują się indeksy wierszy (lub lista takich kolumn w przypadku danych multiindeksowanych), jeżeli plik taką kolumnę zawiera.
- `usecols` - lista, macierz jednowymiarowa; liczby porządkowe lub nazwy kolumn, które chcemy zaimportować do struktury (jeżeli plik takie nazwy kolumn posiada lub nadaliśmy je opcją `names`)
- `squeeze=True/False` - jeżeli `True` a plik zawiera tylko jedną kolumnę, to dane zostaną zrzucone do szeregu `Series`.
- `prefix` - string; prefiks dodawany do automatycznego numeru kolumny w przypadku braku headera w pliku.
- `mangle_dupe_cols=True/False` - jeżeli `True`, to duplikaty kolumn (kolumny o identycznych nazwach) zostaną rozróżnione przy zrzucaniu do struktury. Jeżeli `False`, to dane w kolumnach o zduplikowanych nazwach zostaną nadpisane (opcja dostępna wyłącznie w najnowszych dystrybucjach Pythona).
- `dtype` - słownik; wymuszenie typów danych w kolumnach. Klucze to nazwy kolumn lub ich liczby porządkowe.
- `engine` - `'c'` lub `'python'`; wybór parsera. Parser C jest szybszy, ale w przypadku pliku csv z nietypowymi separatorami lub innymi problemami polecam parser Pythona.
- `converters` - słownik funkcji konwertujących wartości w poszczególnych kolumnach. Klucze to nazwy kolumn lub ich liczby porządkowe.
- `true_values` - lista wartości, które mają być rozumiane jako `True`
- `false_values` - lista wartości, które mają być rozumiane jako `False`
- `skipinitialspace=True/False` - często w plikach csv po przecinku oddzielającym kolumny następuje spacja. Ustawienie `True` spowoduje jej pominięcie przy zrzucaniu danych do kolumn (dane nie będą w postaci `' tekst'`, tylko `'tekst'`.
- `skiprows` - lista, liczba lub funkcja; lista numerów wierszy lub liczba wierszy (licząc od góry), które zostaną pominięte przy imporcie danych. Można również wywołać funkcję (filtr) zwracającą `True/False`. Jeżeli funkcja zwróci `True`, to wiersz zostanie pominięty.
- `skipfooter` - liczba wierszy, które zostaną pominięte, licząc od dołu pliku.
- `nrows` - liczba; liczba wierszy, które zostaną zaimportowane (funkcja przydatna przy zrzucaniu pliku po kawałku).
- `na_values` - lista stringów lub słownik; lista wartości, które mają być rozumiane jako brak danej `NaN`. W przypadku podania słownika, w którym kluczami są nazwy kolumn możemy ustawić osobne listy dla każdej kolumny.
- `keep_default_na=True/False` - jeżeli `True`, to zawsze stringi `‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, ‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, ‘n/a’, ‘nan’, ‘null’` będą rozumiane jako `NaN`.
- `na_filter=True/False` - automatyczne wykrywanie braku danych `NaN` (domyślne i z listy `na_values`).
- `verbose=True/False` - jeżeli `True` w miejscu brakujących danych zostanie wypisane explicite `NaN`.
- `skip_blank_lines=True/False` - jeżeli `True` puste wiersze zostaną pominięte zamiast wypełnienia `NaN`.
- `parse_dates` - `True/False`, lista kolumn (po nazwach lub indeksach), lista list albo słownik. Automatyczne wykrywanie daty:
    - `True` - parser wyszuka daty w nazwach wierszy i kolumn,
    - lista `[...]` indeksów kolumn - elementy tych kolumn zostaną potraktowane jako daty,
    - lista list `[[...]]` indeksów kolumn - kolumny podane w ,,podwójnej'' liście zostaną połączone w jedną kolumnę daty (użyteczne w przypadku, gdy w oryginalnych danych jedna kolumna to dzień, druga miesiąc a trzecia to rok),
    - słownik `{'nazwa':[...]}` dane z kolumn z listy zostaną połączone w datę i powstała kolumna zostanie nazwana `nazwa`.
- `infer_datetime_format=True/False` - format daty zgodny z podanym w pliku (o ile się da) plus szybsze parsowanie daty,
- `keep_date_col=True/False` - jeżeli `True`, to przy parsowaniu daty z kilku kolumn w jedną zachowane zostaną oryginalne kolumny,
- `date_parser` - parser przekształcający tekst w dane. Domyślnie `dateutil.parser.parser`.
- `dayfirst=True/False` - włącza/wyłącza format daty DD/MM,
- `compression` - automatyczne rozpakowywanie pliku przed importem; do wyboru `‘infer’, ‘gzip’, ‘bz2’, ‘zip’, ‘xz’, None`,
- `thousands` - string; postać separatora tysięcznego w importowanych danych,
- `decimal` - string; postać separatora dziesiętnego w importowanych danych; domyślnie `'.'`,
- `lineterminator` - znak końca linii w importowanych danych; działa wyłącznie z parserem C,
- `quotechar` - znak początku/końca cytatu (wszelkie znaki formatujące wewnątrz cytatu zostaną zignorowane)
- `comment` - znak początku komentarza w importowanym pliku. Zawartość linii po tym znaku zostanie zignorowana.
- `encoding` - typ kodowania przy odczycie pliku (domyślnie `'utf-8'`),
- `dialect` - możliwość wyboru jednego ze standardów znaków specjalnych (separator kolumn, cytat itp.) z biblioteki `csv.Dialect`,
- `error_bad_lines=True/False` -  jeżeli `True`, to w przypadku pliku csv z błędnymi wierszami (więcej separatorów kolumnowych niż kolumn) plik nie zostanie zaimportowany do struktury i wyświetli się komunikat błędu. Jeżeli `False`, to błędne wiersze zostaną pominięte i struktura danych zostanie utworzona.
- `warn_bad_lines=True/False` - jeżeli `True` oraz `error_bad_lines=False`, to pojawi się ostrzeżenie o opuszczonych błędnych wierszach.
- `delim_whitespace=True/False` - jeżeli `True`, to spacje i taby będą uważane za separator kolumnowy,
- `low_memory=True/False` - jeżeli `True`, oszczędza pamięć w trakcie parsowania parserem C,
- `memory_map=True/False` - jeżeli podana jest ścieżka dostępu do pliku (nie URL), to dane są zczytywane bezpośrednio z pamięci, z pominięciem klasycznej ścieżki odczytu/zapisu pliku (zaawansowana opcja obsługi olbrzymich zbiorów danych),
- `float_precision` - string `None, 'high', 'round_trip'`; wybór rodzaju konwertera liczb zmiennoprzecinkowych dla parsera C. `None` to domyślny konwerter. Wybór zależy od liczby miejsc po przecinku, jakie chcemy osiągnąć.

Do eksportu danych ze struktury danych (`DataFrame` lub `Series`) do pliku csv używamy funkcji `struktura.to_csv(path_or_buf=None, sep=', ', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression='infer', quoting=None, quotechar='"', line_terminator=None, chunksize=None, date_format=None, doublequote=True, escapechar=None, decimal='.')`, gdzie:

- `path_or_buf` - ścieżka dostępu,
- `sep, decimal, quotechar` - jak wyżej,
- `na_rep` - string; reprezentacja braku danych `NaN`,
- `float_format` - string formatujący zmienne zmiennoprzecinkowe,
- `columns` - lista kolumn do eksportu,
- `header` - `True/False` lub lista stringów; spowoduje zapis nazw kolumn w pierwszej linii pliku csv,
- `index=True/False` - `True` spowoduje zapis do csv kolumny zawierającej indeksy wierszy,
- `index_label` - opcjonalna nazwa dla kolumny indeksów wierszy,
- `mode` - tryb zapisu do pliku, domyślnie `'w'`,
- `encoding` - wybór kodowania,
- `compression` - wybór rodzaju kompresji (jak wyżej),
- `line_terminator` - string; wybór znaku końca linii - domyślnie `os.linesep`, czyli znak odpowiedni dla systemu operacyjnego,
- `chunksize` - liczba lub `None`; liczba wierszy eksportowanych "na raz" (opcja dla dużych zbiorów danych),
- `date_format` - string formatujący dla obiektów typu `datetime`, czyli daty i godziny.

## Import/eksport z arkusza kalkulacyjnego

Odczyt danych z arkusza kalkulacyjnego najczęściej jest dużo mniej kłopotliwy, niż z pliku csv. Używamy do tego funkcji `pandas.read_excel(io, sheet_name=0, header=0, names=None, parse_cols=None, usecols=None, squeeze=False, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, keep_default_na=True, verbose=False, parse_dates=False, date_parser=None, thousands=None, comment=None, skipfooter=0, convert_float=True, mangle_dupe_cols=True)`, gdzie

- `io` - string; ścieżka dostępu do pliku Excela lub URL,
- `sheet_name` - string, liczba lub lista; nazwa (nazwy) lub liczba (liczby) porządkowe arkuszy w pliku, które chcemy zaimportować. Domyślnie 0 (pierwszy arkusz). W przypadku podania większej liczby arkuszy otrzymamy słownik ramek danych.
- `header` - liczba lub lista liczb; numer wiersza zawierającego indeksy kolumn (jak przy imporcie csv)
- `names` - lista nazw kolumn (jak przy imporcie z csv),
- `usecols` - lista nazw lub indeksów kolumn do zaimportowania,
- `squeeze=True/False` - jeżeli arkusz zawiera tylko jedną kolumnę, to zwróci szereg `Series`,
- `dtype` - lista lub słownik wymuszonych typów danych dla kolumn,
- `engine` - do wyboru: `None, 'xldrl'`,
- `converters` - słownik konwerterów typów danych dla poszczególnych kolumn,
- `true_values, false_values, na_values, keep_default_na, verbose` - jak przy imporcie csv,
- `skiprows` - jak przy imporcie csv,
- `nrows` - liczba wierszy parsowanych na raz,
- `parse_dates, date_parser` - jak przy imporcie csv,
- `thousands` - separator tysięczny; używany wyłącznie przy imporcie danych tekstowych jako liczb (dane liczbowe zostaną automatycznie przekonwertowane na format Pythona),
- `comment` - jak przy imporcie csv,
- `skipfooter` - liczba wierszy opuszczanych przy imporcie licząc od dołu,
- `convert_float=True/False` - konwertuje liczby całkowite zapisane w formacie zmiennoprzecinkowym na klasyczny `int`,
- `mangle_dupe_cols=True/False` - jak przy imporcie csv.

Do eksportu struktury danych (`DataFrame`, `Series`) do arkusza Excela służy funkcja `struktura.to_excel(excel_writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True, freeze_panes=None)`, gdzie:

- `excel_writer` - ścieżka dostępu,
- `sheet_name` - string; nazwa utworzonego arkusza,
- `na_rep, float_format` - jak przy eksporcie do csv,
- `columns` - lista kolumn do eksportu,
- `header, index, index_label` - jak przy eksporcie do csv,
- `startrow` - wiersz lewej górnej komórki w arkuszu, od której mają się zaczynać eksportowane dane,
- `startcol` - kolumna lewej górnej komórki w arkuszu, od której mają się zaczynać eksportowane dane,
- `engine` - do wyboru `‘openpyxl’, ‘xlsxwriter’`,
- `merge_cells=True/False` - eksport ramki multiindeksowanej od razu z odpowiednim połączeniem komórek,
- `encoding` - wybór kodowania,
- `inf_rep` - string; reprezentacja nieskończoności (domyślnie `'inf'`),
- `verbose=True/False` - bardziej szczegółowy raport ewentualnych błędów przy eksporcie,
- `freeze_panes` - para liczb; położenie najniższego wiersza i najdalszej kolumny, które mają być zamrożone.

## Zadanie 1.
Zaimportować plik `good.csv` oraz `good1.csv` do struktur danych wyglądających następująco:
```
             klienci    obrót     zysk
data                                 
2019-06-05       20  1034.32   245.93
2019-07-05       32  2054.43   543.23
2019-08-05       19  1042.56   234.99
2019-09-05       24  2450.42   482.24
2019-10-05       42  3010.86   732.13
2019-11-05       53  4529.34  1009.54
2019-05-13       25  1358.60   358.64
2019-05-14       19   934.45   289.13
2019-05-15       36  2874.84   523.49
2019-05-16       22  1873.12   346.97
2019-05-17       36  3010.11   704.21
2019-05-18       48  3722.13   989.06
```

Zapiklować jedną otrzymaną strukturę do pliku `mydlo_powidlo.zip` a drugą wyeksportować do arkusza Excel, plik `mydlo_powidlo.xlsx`, nazwa arkusza `Obroty maj`.

In [135]:
good=pd.read_csv('good.csv',parse_dates={'data':[0,1,2]},dayfirst=True,index_col='data')
good1=pd.read_csv('good1.csv',parse_dates={'data':[0,1,2]},dayfirst=True,index_col='data',lineterminator=";")
print(good)
print(good1)

good.to_pickle('mydlo_powidlo.zip')
good1.to_excel('mydlo_powidlo.xlsx',sheet_name='Obroty maj')

            klienci    obrót     zysk
data                                 
2019-05-06       20  1034.32   245.93
2019-05-07       32  2054.43   543.23
2019-05-08       19  1042.56   234.99
2019-05-09       24  2450.42   482.24
2019-05-10       42  3010.86   732.13
2019-05-11       53  4529.34  1009.54
2019-05-13       25  1358.60   358.64
2019-05-14       19   934.45   289.13
2019-05-15       36  2874.84   523.49
2019-05-16       22  1873.12   346.97
2019-05-17       36  3010.11   704.21
2019-05-18       48  3722.13   989.06
            klienci    obrót     zysk
data                                 
2019-05-06       20  1034.32   245.93
2019-05-07       32  2054.43   543.23
2019-05-08       19  1042.56   234.99
2019-05-09       24  2450.42   482.24
2019-05-10       42  3010.86   732.13
2019-05-11       53  4529.34  1009.54
2019-05-13       25  1358.60   358.64
2019-05-14       19   934.45   289.13
2019-05-15       36  2874.84   523.49
2019-05-16       22  1873.12   346.97
2019-05-17  

## Zadanie 2.
Zaimportować plik `bad.csv` i plik `worse.csv` do struktur danych wyglądających następująco:
```
	      klienci "M,P"  obrót "M,P"  zysk "M,P"
data                                              
2019-06-05             20      1034.32      245.93
2019-07-05             32      2054.43      543.23
2019-08-05             19      1042.56      234.99
2019-09-05             24      2450.42      482.24
2019-10-05             42      3010.86      732.13
2019-11-05             53      4529.34     1009.54
2019-05-13             25      1358.60      358.64
2019-05-14             19       934.45      289.13
2019-05-15             36      2874.84      523.49
2019-05-16             22      1873.12      346.97
2019-05-17             36      3010.11      704.21
2019-05-18             48      4722.13     1289.06
```

In [133]:
bad=pd.read_csv('bad.csv',header=1 ,parse_dates={'data':[0,1,2]},dayfirst=True,usecols=[0,1,2,3,4,5],index_col='data',names=['d1','d2','d3','klienci "M,P"','obrót "M,P"','zysk "M,P"'],comment='%')
bad

,"klienci ""M,P""","obrót ""M,P""","zysk ""M,P"""
data,,,
2019-05-06,20,1034.32,245.93
2019-05-07,32,2054.43,543.23
2019-05-08,19,1042.56,234.99
2019-05-09,24,2450.42,482.24
2019-05-10,42,3010.86,732.13
2019-05-11,53,4529.34,1009.54
2019-05-13,25,1358.60,358.64
2019-05-14,19,934.45,289.13
2019-05-15,36,2874.84,523.49


In [148]:
worse=pd.read_csv('worse.csv',sep="-",error_bad_lines=False,header=1 ,parse_dates={'data':[0,1,2]},dayfirst=True,index_col='data',names=['d1','d2','d3','klienci "M,P"','obrót "M,P"','zysk "M,P"'],comment='%',decimal=",",thousands=" ")
worse

/tmp/ipykernel_10202/463445861.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  worse=pd.read_csv('worse.csv',sep="-",error_bad_lines=False,header=1 ,parse_dates={'data':[0,1,2]},dayfirst=True,index_col='data',names=['d1','d2','d3','klienci "M,P"','obrót "M,P"','zysk "M,P"'],comment='%',decimal=",",thousands=" ")
Skipping line 10: expected 6 fields, saw 9
Skipping line 14: expected 6 fields, saw 10
Skipping line 16: expected 6 fields, saw 13



,"klienci ""M,P""","obrót ""M,P""","zysk ""M,P"""
data,,,
2019-05-06,20,1034.32,245.93
2019-05-07,32,2054.43,543.23
2019-05-08,19,1042.56,234.99
2019-05-09,24,2450.42,482.24
2019-05-10,42,3010.86,732.13
2019-05-11,53,4529.34,1009.54
2019-05-13,25,1358.60,358.64
2019-05-14,19,934.45,289.13
2019-05-15,36,2874.84,523.49


## Zadanie 3.
Zaimportować plik `disaster.csv` do struktury wyglądającej jak powyżej. Gotową strukturę wyeksportować do pliku `mydlo_powidlo.csv`. Zaimportować ją z powrotem i sprawdzić typy zmiennych w kolumnach.

In [147]:
disaster=pd.read_csv('disaster.csv',sep="&", header=1,
                     usecols=[0,1,2,4,6,7],
                     parse_dates={'data':[0,1,2]},dayfirst=True,
                     index_col='data',comment="/",
                     skiprows=[10,17,18],
                    decimal=",",thousands=" ")
print(disaster)

disaster.to_csv('mydlo_powidlo.csv')

             klienci "M,P"   obrót "M,P"   zysk "M,P"
data                                                 
2019-05-06              20       1034.32       245.93
2019-05-07              32       2054.43       543.23
2019-05-08              19       1042.56       234.99
2019-05-09              24       2450.42       482.24
2019-05-10              42       3010.86       732.13
2019-05-11              53       4529.34      1009.54
2019-05-13              25       1358.60       358.64
2019-05-14              19        934.45       289.13
2019-05-15              36       2874.84       523.49
2019-05-16              22       1873.12       346.97
2019-05-17              36       3010.11       704.21
2019-05-18              48       4722.13      1289.06


In [149]:
disaster= pd.read_csv('mydlo_powidlo.csv')
disaster.dtypes

data               object
 klienci "M,P"      int64
 obrót "M,P"      float64
 zysk "M,P"       float64
dtype: object

## Zadanie 4.
Zaimportować z pliku `grupa00.xlsx` obydwa arkusze:
- każdy arkusz do osobnej struktury,
- obydwa arkusze razem do słownika struktur.

Odpowiednio wybrać kolumnę numerującą wiersze, aby uniknąć niepotrzebnej podwójnej numeracji.

In [151]:
sem1 = pd.read_excel('grupa00.xlsx','Semestr I',index_col=0)
sem2 = pd.read_excel('grupa00.xlsx','Semestr II',index_col=0)
sem1,sem2

(    Imię    Nazwisko  kol1  kol2  kol3  suma
 lp                                          
 1   Anna     Borówka   7.0   3.0   5.5  15.5
 2    Jan      Brokuł   6.0   2.0   3.0  11.0
 3    Jan      Cebula   4.5   2.0   7.5  14.0
 4   Anna     Cukinia   2.0   4.6   3.5  10.1
 5   Anna       Dynia   3.0   5.0   8.0  16.0
 6    Jan      Jarmuż   4.5   2.5   6.5  13.5
 7   Anna    Kalafior   6.0   6.0   3.0  15.0
 8    Jan     Kapusta   8.5   3.5   2.5  14.5
 9    Jan   Marchewka   3.0   8.0   4.5  15.5
 10  Anna      Oliwka   3.5   7.5   6.0  17.0
 11  Anna   Pasternak   0.0   3.5   2.0   5.5
 12   Jan  Pietruszka   NaN   4.0   1.5   5.5
 13  Anna         Por   6.0   2.5   1.0   9.5
 14   Jan      Sałata   4.5   4.0   0.0   8.5
 15   Jan     Tatarak   2.0   5.0   3.5  10.5
 16  Anna    Ziemniak   3.0   5.5   7.0  15.5,
     Imię    Nazwisko  kol1  kol2  kol3  suma
 lp                                          
 1   Anna     Borówka   3.0   3.0   5.5  11.5
 2    Jan      Brokuł   6.0   2.5

In [152]:
sem1_2 = pd.read_excel('grupa00.xlsx',index_col=0,sheet_name=[0,1])
sem1_2

{0:     Imię    Nazwisko  kol1  kol2  kol3  suma
 lp                                          
 1   Anna     Borówka   7.0   3.0   5.5  15.5
 2    Jan      Brokuł   6.0   2.0   3.0  11.0
 3    Jan      Cebula   4.5   2.0   7.5  14.0
 4   Anna     Cukinia   2.0   4.6   3.5  10.1
 5   Anna       Dynia   3.0   5.0   8.0  16.0
 6    Jan      Jarmuż   4.5   2.5   6.5  13.5
 7   Anna    Kalafior   6.0   6.0   3.0  15.0
 8    Jan     Kapusta   8.5   3.5   2.5  14.5
 9    Jan   Marchewka   3.0   8.0   4.5  15.5
 10  Anna      Oliwka   3.5   7.5   6.0  17.0
 11  Anna   Pasternak   0.0   3.5   2.0   5.5
 12   Jan  Pietruszka   NaN   4.0   1.5   5.5
 13  Anna         Por   6.0   2.5   1.0   9.5
 14   Jan      Sałata   4.5   4.0   0.0   8.5
 15   Jan     Tatarak   2.0   5.0   3.5  10.5
 16  Anna    Ziemniak   3.0   5.5   7.0  15.5,
 1:     Imię    Nazwisko  kol1  kol2  kol3  suma
 lp                                          
 1   Anna     Borówka   3.0   3.0   5.5  11.5
 2    Jan      Brokuł   6.0